In [3]:
SEMINAR_PATH = '/home/jim/PycharmProjects'
import sys
sys.path.append(SEMINAR_PATH)

import seminar
import seminar.preproc.readers as rs
import seminar.preproc.io as io

from seminar.preproc.readers import BBox
from seminar.learn.utils import pred_array
from seminar.preproc.transformers import FFT_SAR_timeseries, VV_VH_Combo, _FixedCombo
from seminar.preproc.io import read_raster, align
from seminar.learn.cnn_lstm import convlstm, convgru

from fiona.crs import from_epsg

import os
import re
import xarray as xr
import pandas as pd
import numpy as np
import random
import rasterio as rio
from functools import partial
import itertools

from operator import itemgetter
from pympler import tracker

from scipy import signal
import scipy.fftpack as ff
from scipy.fft import fftshift
from scipy.optimize import linear_sum_assignment

import sklearn.cluster as cl
import sklearn.svm as svm
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler, FunctionTransformer, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import matplotlib as mpl

import geoviews as gv
import geoviews.feature as gf

import dill

import hvplot.xarray
import hvplot.pandas
gv.extension('bokeh')


ModuleNotFoundError: No module named 'seminar'

In [2]:
from importlib import reload
reload(seminar)
reload(seminar.preproc)


<module 'seminar.preproc' from '/home/jim/PycharmProjects/seminar/preproc/__init__.py'>

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Preprocessing Data

### Read

In [4]:
bbox = BBox((5.905e5, 4.835e6, 6.6997e5, 4.798e5), crs=from_epsg(32631))

In [5]:
# Read in data, crop first, then load from dir with cropped
reader_vv = rs.SeminarReader(time_pattern=r'_([0-9]+)_', 
                             incl_pattern='.*',
                             dirpath='/home/jim/PycharmProjects/seminar/notebooks/data/vv_6d_12d_cropped')
                             #dirpath='/home/jim/mount/shared/course/geo441/data/2020_Camargue/s1/composite/iw/6d_12d/vv')

arrs_vv, bboxs_vv = reader_vv.query() 
                                  #time=(reader_vv.min_time, reader_vv.min_time + timedelta(days=20)), 
                                  #out=True, out_dir='/home/jim/PycharmProjects/seminar/notebooks/data/test_out')

In [ ]:
reader_vh = rs.SeminarReader(time_pattern=r'_([0-9]+)_', 
                             incl_pattern='.*',
                             dirpath='/home/jim/PycharmProjects/seminar/notebooks/data/vh_6d_12d_cropped')

#arrs_vh, bboxs_vh = reader_vh.query(out=True, out_dir='/home/jim/PycharmProjects/seminar/notebooks/data/vh_cropped', bbox=bbox)
arrs_vh, bboxs_vh = reader_vh.query()

In [ ]:
arrs, bboxs = io.read_raster('/home/jim/PycharmProjects/seminar/notebooks/data/gt2/query_out/pnrc_mos_2016_c_rasterized30x30m_UTM31N_cropped.tif')
                            # '/home/jim/Dropbox/RS-Camargue-Box/Klassierung_2/pnrc_mos_2016_c_rasterized30x30m_UTM31N.tif', 
                            # bbox=bbox, out_dir='/home/jim/PycharmProjects/seminar/notebooks/data/gt2', out=True)
ground_truth_classif = arrs[0]
ground_truth_classif.data = ground_truth_classif.astype(rio.int8)

### Align ground truth to data

In [ ]:
# Align ground truth to input data
aligned_gt = align(arrs_vv[0], ground_truth_classif, '/home/jim/PycharmProjects/seminar/notebooks/data/algined_ground_truth.tif')

In [ ]:
uniques = np.unique(aligned_gt)
mapping = {code: val for val, code in enumerate(uniques)}
mapping[0] = -1

mapper = np.vectorize(lambda entry: mapping.get(entry, entry))
aligned_gt.data = mapper(aligned_gt.data).astype(np.int8)

In [10]:
aligned_gt.hvplot.image(rasterize=True)

:DynamicMap   []
   :Image   [x,y]   (value)

In [ ]:
gt = pd.DataFrame(aligned_gt.data.flatten())

In [ ]:
valid_gt = np.where(gt > -1)
only_valid_gt = gt.iloc[valid_gt[0]]

### to pandas

#### VV

In [ ]:
xx_unnorm = arrs_vv.data.reshape(arrs_vv.shape[0], -1).transpose()
dfxx_unnorm = pd.DataFrame(xx_unnorm, columns=arrs_vv.time.data)

In [ ]:
# there are invalid pixels
no_invalids = np.where(np.count_nonzero(dfxx_unnorm, axis=1) == xx_unnorm.shape[1])
dfxx_unnorm = dfxx_unnorm.iloc[no_invalids]

# there is a region with corrupted data
no_corrupted = np.where(np.count_nonzero(dfxx_unnorm < .5e-4, axis=1) < 4)
only_valid_vv = dfxx_unnorm.iloc[no_corrupted]

#### VH

In [ ]:
xx_unnorm = arrs_vh.data.reshape(arrs_vv.shape[0], -1).transpose()
dfxx_unnorm = pd.DataFrame(xx_unnorm, columns=arrs_vh.time.data)

In [ ]:
# there are invalid pixels
no_invalids = np.where(np.count_nonzero(dfxx_unnorm, axis=1) == xx_unnorm.shape[1])
dfxx_unnorm = dfxx_unnorm.iloc[no_invalids]

# there is a region with corrupted data
no_corrupted = np.where(np.count_nonzero(dfxx_unnorm < .5e-4, axis=1) < 4)
only_valid_vh = dfxx_unnorm.iloc[no_corrupted]

In [ ]:
(only_valid_vh.iloc[1090, :]).hvplot() * (only_valid_vv.iloc[1090, :]).hvplot()

In [ ]:
only_valid_vv.shape

### sample

In [ ]:
intersect = only_valid_vv.index.intersection(only_valid_vh.index).intersection(only_valid_gt.index)
only_valid_dfxx_unnorm = pd.concat((only_valid_vv.loc[intersect], only_valid_vh.loc[intersect]), axis=1)

In [ ]:
len(only_valid_dfxx_unnorm)

In [ ]:
# Remove heavy memory
xx_unnorm = 0
only_valid_vh = 0
only_valid_vv = 0
dfxx_unnorm = 0
index = 0
no_invalids = 0
no_corrupted = 0

In [21]:
# sample from dataset
samples = only_valid_dfxx_unnorm.sample(n=100000)

### Build Network

In [1]:
lstm = convlstm.ConvLSTM(in_channels=2, num_layers=5, hidden_channels=[5, 4, 3, 2, 1], kernel_size=[3]*5)

NameError: name 'convlstm' is not defined